### Настройка окружения

Для начала вам необходимо выполнить ряд команд чтобы настроить окружение для дальнейшей работы, это позволит первое время не заниматься долгим деплоем, а сразу начать писать код и работать с airflow.

In [ ]:
# Установка Airflow
!pip install apache-airflow==2.4.2
!pip install apache-airflow-providers-http
!pip install apache-airflow-providers-telegram

!airflow db init

In [ ]:
# Создадим папку dags
# В этой папке лежат скрипты для создания дагов
# Это стандартное имя для  данной папки
!mkdir /root/airflow/dags
!touch /root/airflow/dags/dag.py

In [ ]:
!airflow users create \
          --username admin \
          --firstname admin \
          --lastname admin \
          --role Admin \
          --email admin@example.org \
          -p 12345

In [ ]:
# Включим веб-сервер
!airflow webserver -p 18273 -D

In [ ]:
# Запуск шедулера
!airflow scheduler -D

In [ ]:
# Последующие команды не имеют отношения к Airflow
# Они нужни только для корректной работы веб морды
# в среде Google Colab

!pip install pyngrok
!ngrok authtoken  # найти его можно https://dashboard.ngrok.com/get-started/setup 

# Эта команда просто отображет веб морду на другой адрес
# Его вы можете найти https://dashboard.ngrok.com/cloud-edge/status
# При каждом отключении ссылка будет меняться
!nohup ngrok http 18273 > /dev/null &

### Задача на разработку

Необходимо реализовать PythonOperator который будет бросать исключение raise AirflowException . После этого реализуйте функцию которая будет отправлять сообщение в Telegram при статусе задачи failed, вам нужно создать свой канал и бота которые будет отсылать сообщение.

In [ ]:
from airflow import DAG
from datetime import timedelta, datetime
from airflow.utils.dates import days_ago
from airflow.providers.telegram.operators.telegram import TelegramOperator
from airflow.operators.python import PythonOperator
from airflow.exceptions import AirflowException

def on_failure_callback(context):
    send_message = TelegramOperator(
        task_id='send_message_telegram',
        telegram_conn_id='telegram_id',
        chat_id='', #id канала
        text='oh no!',
        dag=dag)
    return send_message.execute(context=context)

def exception():
  raise AirflowException('send error to telegram')

# Создадим объект класса DAG
dag = DAG(
    'dag',
    start_date=datetime(2015, 12, 1),
    on_failure_callback=on_failure_callback,
    schedule_interval='@daily')

wrong_task = PythonOperator(task_id='error', python_callable=exception, dag=dag)

wrong_task

<ipython-input-7-34ca26cc0e2a>:22 RemovedInAirflow3Warning: Param `schedule_interval` is deprecated and will be removed in a future release. Please use `schedule` instead.